### kernel density estimation and kernel regression for prediction
This notebook is my first attempt to write down the ddiff and dddiff models that I have been working on

mixed recursive/iterative approach (rather than vectorized approach that is harder to generalize
- create separate submodules for KDE and KDreg
    - develop a system for storing differenced but unweighted data and the appropriate mask to lessen memory usage. 
        -1's or 0's depending on i/j/k/l/.... values
    - figure out the deepest level requested by the user and levels that are less deep are just slices of the deeper model. 
        -scout out the tree to build the differenced dataset. 
          
        
 plot and compare
on synthetic data, 1, 2, 3+ mixed distributions
1d - Ndiff vs gaussian kernel vs kernel_tunneling
2d -  


multidimensional x problem
e.g., parameter treatment
"product kernel approach" vs l2 "el two" (radial basis?)distance 

In [1]:
import numpy as np

### generate a simple linear dataset y=xb+e

In [2]:
import data_gen as dg
data1=dg.data_gen(data_shape=(400,1),seed=0)
y=data1.y
xfull=data1.x
x=data1.x[:,1:] #drop first column of x (all 1's)
e=data1.e
n,p=x.shape

### create modeldict
- 'Ndiff_type': refers to the mathematical form of Ndiff
  - 'product'-Ndiff1 multiplied by Ndiff2
  - 'recursive'-Ndiff1's bw is Ndiff 2
- max_bw_Ndiff: is the depth of Ndiffs applied in estimating the bandwidth.
- 'normalize_Ndiffwtsum':
  - 'across' means sum across own level of kernelized-Ndiffs and divide by that sum (CDF approach)
  - 'own_n' means for (n+k)diff where n+K=max_bw_Ndiff
- Ndiff_bw_kern:
  - rbfkern means use the radial basis function kernel
  - 'product' means use product kernel like as in liu and yang eq1. 
- 'regression_model':
  - 'NW' means use nadaraya-watson kernel regression
  - 'full_logit' means local logit with all variables entering linearly
  - 'rbf_logit' means local logit with 1 parameters: scaled l2 norm centered on zero 
 (globally or by i?). Is this a new idea?
- outer_x_bw_form: if x is separated into blocks of rvars that are combined within a block using rbf kernel, 
  - 'one_for_all' - use same hx bw for all blocks
  - 'one_per_block' - each block or rvars gets an hx
- ykern_grid
  - if int, then create int evnely spaced values from -3 to 3 (standard normal middle ~99%)
  - 'no' means use original data points (self is masked when predicting self)
- xkern_grid
  - much like ykern_grid, but used less frequently since x's are typically pre-specified
- product_kern_norm: when multiplying kernels across random variables, this parameter determines if each random variable has its kernels normalized before the product or not
  - 'self' means each random variable has its kernels divided by the sum of kernels across that random variable (as if creating a cdf)
  - 'none' means no normalization prior to taking products across rvar kernels
- hyper_param_form_dict is a nested dictionary
  - Ndiff_exponent is the exponent wrapped around typically a sum of kernels for each Ndiff level 
after level 1 (i.e., (i-j) or centered, obvious/conventional level.)
  - x_bandscale is the parameter specific to each variable (each x in X) used for prediction (y)
  - Ndiff_depth_bw is used as the kernel's bandwidth (h) at each level of Ndiff including at level 1 
  - outer_x_bw vanilla bandwidth for the rbf or product kernel
  - outer_y_bw vanilla bandwidth for y

In [3]:
max_bw_Ndiff=1
modeldict1={
    'Ndiff_type':'product',
    'max_bw_Ndiff':max_bw_Ndiff,
    'normalize_Ndiffwtsum':'own_n',
    'xkern_grid':'no',
    'ykern_grid':'no',
    'outer_kern':'gaussian',
    'Ndiff_bw_kern':'rbfkern',
    'outer_x_bw_form':'one_for_all',
    'regression_model':'NW',
    'product_kern_norm':'self',
    'hyper_param_form_dict':{
        'Ndiff_exponent':'free',
        'x_bandscale':'non-neg',
        'Ndiff_depth_bw':'non-neg',
        'outer_x_bw':'non-neg',
        'outer_y_bw':'non-neg',
        'y_bandscale':'fixed'
        }
    }

#modeldict1['ykern_grid']=17#override


#set starting or fixed values for hyper parameters
hyper_paramdict1={
    'Ndiff_exponent':.05*np.ones([max_bw_Ndiff-1,]),
    'x_bandscale':np.ones([p,]),
    'outer_x_bw':np.array([0.5,]),
    'outer_y_bw':np.array([0.5,]),
    'Ndiff_depth_bw':np.ones([max_bw_Ndiff,])*0.15,
    'y_bandscale':np.ones([1,])
    }
#assert len(hyper_paramdict1['Ndiff_exponent'])==modeldict1['max_bw_Ndiff'],'Ndiff_exponent does not match ' \
#                                                                            'depth of max_bw_Ndiff'
assert len(hyper_paramdict1['Ndiff_depth_bw'])==modeldict1['max_bw_Ndiff'],'Ndiff_depth_bw does not match ' \
                                                                             'depth of max_bw_Ndiff'
#create optimization dictionary
optimizedict1={
    'method':'Nelder-Mead',
    'hyper_param_dict':hyper_paramdict1,
    'model_dict':modeldict1
    }

#-----------------Calibrate/Optimize--------

In [4]:
import mykern as mk
#y=np.linspace(0,100,n)#what happens if y has no relationship to x
optimized_Ndiff_kernel=mk.optimize_free_params(y,x,optimizedict1)


1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,

In [5]:
print(optimized_Ndiff_kernel.mse)
import myreg
just_linreg=myreg.reg(y,xfull)
just_linreg.myreg()
print('linreg_mse:',just_linreg.mse)

 final_simplex: (array([[-19.20750705,   2.729048  ,  -3.80411326,  19.70440456],
       [-19.20754889,   2.72905337,  -3.80412504,  19.70444583],
       [-19.20747417,   2.72904598,  -3.80411561,  19.70437993],
       [-19.20759259,   2.72906039,  -3.80413935,  19.70449113],
       [-19.20752898,   2.72905372,  -3.8041311 ,  19.70443376]]), array([0.00516618, 0.00516618, 0.00516618, 0.00516618, 0.00516618]))
           fun: 0.005166182874563142
       message: 'Optimization terminated successfully.'
          nfev: 176
           nit: 81
        status: 0
       success: True
             x: array([-19.20750705,   2.729048  ,  -3.80411326,  19.70440456])
linreg_mse: 1.4169110282095976


In [6]:
from bokeh.io import show, output_notebook#,curdoc,save, output_file
from bokeh.plotting import figure
#from bokeh.models import ColumnDataSource, Range1d, BBoxTileSource
#from bokeh.layouts import row
output_notebook()

Loading BokehJS ...

In [7]:
#xgrid=np.linspace(1,n,n)

xgrid=x[:,0]
yhat=optimized_Ndiff_kernel.yhat
p=figure(title='y and yhat', plot_width=900, plot_height=500)
p.xaxis.axis_label = 'observations'
p.scatter(xgrid,y,size=10,color='blue',alpha=0.4,legend='y')
#p.line(xgrid,precip,color='blue',alpha=0.6,legend='precipitation')
p.scatter(xgrid,yhat,size=5,color='green',alpha=0.9,legend='yhat')
p.circle(xgrid,just_linreg.yhat,color='red',legend='just lin reg yhat')
p.legend.location = "top_left"
p.yaxis.visible=False
show(p)

In [8]:
yhat.shape

(40,)

In [9]:
print(yhat)

[9.600626283961752 8.893701732165685 10.144142088419951 9.156445419048081
 14.676879161731543 12.930727506470152 11.1378762865976 10.948316725845316
 7.996739683023892 12.473149400025495 16.700461459023074 8.811852910674252
 10.070144681715737 9.018872589447579 12.80610546389439 7.65104191248491
 12.522556268331181 8.580692494575269 9.39054059608906 6.36096174107533
 6.627010981550468 7.676639674166172 7.757160818325932 3.471390161627947
 11.319347761994672 7.057231062149816 15.32282005689655 11.255246946723746
 14.24720207377515 8.658243460782415 12.677753248097602 12.934273070292647
 12.22030419992346 13.216682097018928 7.527986499496455 12.01438821174965
 5.499248034977371 10.72006749018447 9.799028588113908 8.610007109215488]


In [10]:
print(y)

[ 9.58494756  8.85989673 10.14239966  9.12937744 14.79136076 13.00043599
 11.16161423 10.96719416  7.93993566 12.53112511 16.86682979  8.77594923
 10.06650489  8.9882771  12.87261851  7.58537384 12.58179882  8.53886162
  9.36947506  6.26221469  6.53508571  7.61162796  7.69421375  3.29855153
 11.34773882  6.97633707 15.45386425 11.28199439 14.35066632  8.61840107
 12.74097521 13.00407247 12.2717967  13.29372275  7.45916316 12.06060082
  5.37840576 10.73309238  9.7884371   8.56892789]


In [11]:
optimized_Ndiff_kernel.fixed_or_free_paramdict

{'Ndiff_exponent': {'fixed_or_free': 'free',
  'const': 'free',
  'location_idx': (0, 0)},
 'x_bandscale': {'fixed_or_free': 'free',
  'const': 'non-neg',
  'location_idx': (0, 1)},
 'Ndiff_depth_bw': {'fixed_or_free': 'free',
  'const': 'non-neg',
  'location_idx': (1, 2)},
 'outer_x_bw': {'fixed_or_free': 'free',
  'const': 'non-neg',
  'location_idx': (2, 3)},
 'outer_y_bw': {'fixed_or_free': 'free',
  'const': 'non-neg',
  'location_idx': (3, 4)},
 'y_bandscale': {'fixed_or_free': 'fixed',
  'const': 'fixed',
  'location_idx': (0, 1)},
 'free_params': array([-19.20759259,   2.72906039,  -3.80413935,  19.70449113]),
 'fixed_params': array([1.])}

In [12]:
np.ma.count_masked(yhat)

0